In [1]:
!pip install dfply
!pip install plotly
import pandas as pd
import numpy as np
import math
from dfply import *
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import statistics 
import plotly.graph_objects as go
from scipy import stats
from scipy.stats import pearsonr
import plotly.graph_objects as go
import warnings
from scipy.stats import shapiro
from datetime import date, timedelta

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [0]:
trophies = pd.read_pickle("/content/drive/My Drive/Capstone/Data/Clean/trophies_Clean.pkl")
players = pd.read_pickle("/content/drive/My Drive/Capstone/Data/Clean/Players_Clean.pkl")
transfers = pd.read_pickle("/content/drive/My Drive/Capstone/Data/Clean/Transfers_Clean.pkl")
transfers = transfers[(transfers.loan == False) & (transfers.fee > 0)]

In [0]:
trophies["date"] = trophies.index
trophies.reset_index(inplace = True, drop = True)
trophies["year"] = trophies["date"].apply(lambda x : x.year)
trophies.drop_duplicates(inplace = True)
trophies_tm = pd.merge(trophies,players[['tm_id','sofifa_id']],on = ["sofifa_id"])
trophies_tm.drop("sofifa_id", axis = "columns", inplace = True)

In [0]:
trophies_tm = trophies_tm[trophies_tm.tm_id.isin(transfers.tm_id.unique())]

In [0]:
trophies_tm = trophies_tm.groupby(["tm_id","trophy","year"]).apply(lambda x: x.trophy.count()).reset_index()

In [0]:
trophies_tm.columns = ["tm_id","trophy","year","count"]

In [7]:
trophies_tm

,tm_id,trophy,year,count
0,532,Runner-up,2000,1
1,532,Runner-up,2002,1
2,532,Runner-up,2004,1
3,532,Runner-up,2006,1
4,532,Runner-up,2008,2
...,...,...,...,...
17297,48759594,Winner,2009,1
17298,48759594,Winner,2010,1
17299,48759594,Winner,2016,1
17300,48759594,Winner,2017,1


In [0]:
trophies_wide = pd.pivot_table(trophies_tm, values='count', index=['tm_id', 'year'],
                    columns='trophy', aggfunc=np.sum)
trophies_wide.columns = trophies_wide.columns.astype(list)
trophies_wide.reset_index(inplace = True, drop = False)  

In [0]:
trophies_wide.fillna(0, inplace = True)

In [10]:
trophies_wide.columns = ["tm_id","year","runner_up","winner"]
trophies_wide.dtypes

tm_id          int64
year           int64
runner_up    float64
winner       float64
dtype: object

In [0]:
trophies_wide = trophies_wide.groupby(['tm_id']).apply(lambda x : x.sort_values("year")).reset_index(drop = True)

In [0]:
trophies_wide["cum_runner_up"] = trophies_wide.groupby("tm_id").apply(
    lambda df : df["runner_up"].cumsum() - df["runner_up"]).values
trophies_wide["cum_winner"] = trophies_wide.groupby("tm_id").apply(
    lambda df : df["winner"].cumsum() - df["winner"]).values

In [0]:
transfers_trophies = pd.merge(transfers,trophies_wide, how = "left", on = ["tm_id","year"])

In [0]:
transfers_trophies["runner_up"].fillna(0,inplace = True)
transfers_trophies["winner"].fillna(0,inplace = True)
transfers_trophies["cum_runner_up"].fillna(0,inplace = True)
transfers_trophies["cum_winner"].fillna(0,inplace = True)
transfers_trophies = transfers_trophies.dropna() 

In [15]:
transfers_trophies

,date,tm_id,from,to,fee,mv,season,loan,year,runner_up,winner,cum_runner_up,cum_winner
0,2011-07-01,17259,FC Schalke 04,Bayern Munich,30000000.0,28000000.0,11/12,False,2011.0,2.0,1.0,3.0,1.0
1,2015-07-01,40680,VfB Stuttgart,Bayern Munich,3500000.0,3500000.0,15/16,False,2015.0,0.0,1.0,1.0,0.0
2,2019-07-01,281963,Atlético Madrid,Bayern Munich,80000000.0,70000000.0,19/20,False,2019.0,2.0,1.0,3.0,5.0
3,2017-07-01,166601,TSG Hoffenheim,Bayern Munich,20000000.0,25000000.0,17/18,False,2017.0,0.0,1.0,2.0,0.0
4,2007-08-21,26485,Hertha BSC,Hamburger SV,1100000.0,750000.0,07/08,False,2007.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
11769,2013-01-14,81800,FK Jablonec,Viktoria Plzen,640000.0,900000.0,12/13,False,2013.0,0.0,0.0,0.0,0.0
11770,2009-07-01,81800,Slavia Prague,FK Jablonec,200000.0,250000.0,09/10,False,2009.0,0.0,0.0,0.0,0.0
11771,2017-07-01,147540,Sparta Praha,Viktoria Plzen,1000000.0,1250000.0,17/18,False,2017.0,0.0,0.0,0.0,0.0
11772,2018-09-06,384856,SFC Opava,Viktoria Plzen,600000.0,450000.0,18/19,False,2018.0,0.0,0.0,0.0,0.0


In [0]:
transfers_trophies.to_pickle("/content/drive/My Drive/Capstone/Data/Clean/transfers_trophies.pkl")

In [0]:
markval = pd.read_pickle("/content/drive/My Drive/Capstone/Data/Clean/markval_detailed.pkl")

In [17]:
markval = markval.groupby(['tm_id']).apply(lambda x : x.sort_values("year")).reset_index(drop = True)

KeyboardInterrupt: ignored

In [0]:
markval["cum_mv"] = markval.groupby("tm_id").apply(lambda markval : markval["mv"].rolling(2).mean()).values

In [0]:
markval["cum_mv"].fillna(0, inplace = True)

In [0]:
markval.to_pickle("/content/drive/My Drive/Capstone/Data/Clean/markval_detailed.pkl")